In [ ]:
from lib.models import RecommendSystemModel

from typing import List, Any, Tuple
from numpy.typing import NDArray

In [1]:
import functools
def update_class(
    main_class=None, exclude=("__module__", "__name__", "__dict__", "__weakref__")
):
    """Class decorator. Adds all methods and members from the wrapped class to main_class

    Args:
    - main_class: class to which to append members. Defaults to the class with the same name as the wrapped class
    - exclude: black-list of members which should not be copied
    """

    def decorates(main_class, exclude, appended_class):
        if main_class is None:
            main_class = globals()[appended_class.__name__]
        for k, v in appended_class.__dict__.items():
            if k not in exclude:
                setattr(main_class, k, v)
        return main_class

    return functools.partial(decorates, main_class, exclude)

In [2]:
class MyClass:
    def method1(self):
        print("method1")
me = MyClass()


In [3]:
@update_class()
class MyClass:
    def method2(self):
        print("method2")
me.method1()
me.method2()

method1
method2


In [4]:
# from lib.utils.decorator import update_class
import functools
from lib.models import RecommendSystemModel

from typing import List, Any, Tuple
from numpy.typing import NDArray

ModuleNotFoundError: No module named 'lib.utils.decorator'

In [ ]:
class SVDModel(RecommendSystemModel):
    def __init__(self) -> None:
        super().__init__()
svd = SVDModel()

In [ ]:
@update_class()
class SVDModel(RecommendSystemModel):
    def split(ratio: float, tensor: bool = False) -> List[NDArray]:
        print(123)
        # return super().split(ratio, tensor)

In [ ]:
@update_class()
class SVDModel(RecommendSystemModel):
    def data_loader(self) -> None:
        print(456)
        # return super().data_loader()

In [ ]:
@update_class()
class SVDModel(RecommendSystemModel):
    def learn_to_recommend(self, data: Any, features: int = 10, lr: float = 0.0002, epochs: int = 101, weight_decay: float = 0.02, stopping: float = 0.001) -> Tuple[NDArray, NDArray, float, float]:
        print(789)
        # return super().learn_to_recommend(data, features, lr, epochs, weight_decay, stopping)

In [ ]:
@update_class()
class SVDModel(RecommendSystemModel):
    def prediction(P: NDArray, Q: NDArray, u: int, i: int) -> float:
        print(321)
        # return super().prediction(P, Q, u, i)

In [ ]:
@update_class()
class SVDModel(RecommendSystemModel):
    def loss(data: Any, P: NDArray, Q: NDArray) -> float:
        print(654)
        # return super().loss(data, P, Q)

In [ ]:
@update_class()
class SVDModel(RecommendSystemModel):
    def svd(self):
        print(987)
        # return super().svd()